In [651]:
import argparse
import bz2
import xml.etree.ElementTree as ET
import pandas as pd

In [652]:
def get_page(wiki_file):

    page_lines = []
    page_started, page_ended = False, False
    
    #print(wiki_file)

    while not page_started:
        line = wiki_file.readline()
        if line == '':
            return False
        if line.find('<page>') != -1:
            page_started = True
            page_lines.append(line)

    while not page_ended:
        line = wiki_file.readline()
        if line.find('</page>') != -1:
            page_ended = True
        page_lines.append(line)

    return ''.join(page_lines)


def parse_wiki(input_path):
    import_data_pd = pd.DataFrame()
    
    wiki_file = bz2.open(input_path,mode='t',encoding="UTF8")

    category_list = []
    text_list = []
    count = 0
    
    while True:
        if count%10000 == 0:
            print(count)
            if count == 200000:
                break
        
        page_str = get_page(wiki_file)
        #print(page_str)
        if not page_str:
            break

        # XMLをパースする
        root = ET.fromstring(page_str)

        title = root.find('title').text
        if title in company_list:
            print(title)
            
            gaiyou_str = get_gaiyou(page_str)
            if gaiyou_str != "temp":
                category = company_pd["category_od"][company_pd["company"] == title].values[0]
                category_list.append(int(category))
                text_list.append(clean_text(gaiyou_str))
        count += 1
                
    import_data_pd["category"] = category_list
    import_data_pd["text"] = text_list
    
    return import_data_pd
                
                
def get_gaiyou(page_str):
    page_lines = []
    company_flag, gaiyou_start, gaiyou_end = False, False, False
    line = "temp"

    if page_str.find('|社名') != -1 or page_str.find('| 社名') != -1:
        
        num = page_str.find('== 概要 ==')
        if num != -1:
            temp_str = page_str[num+10::]
            line = temp_str[:temp_str.find('=='):]
            
    return line

In [653]:
category = pd.read_csv("category.csv",encoding="SJIS",sep="\t",dtype = {"category":"object"})

In [654]:
company_pd = pd.read_csv("company.csv",encoding="UTF8",sep="\t",dtype={"category":object})

In [655]:
import category_encoders as ce
oe = ce.OrdinalEncoder(cols="category", handle_unknown='ignore')
company_pd_oe = oe.fit_transform(company_pd)
company_pd["category_od"] = company_pd_oe["category"] - 1

In [656]:
company_list = list(company_pd["company"])
wiki_file = "jawiki-latest-pages-articles.xml.bz2"

import_data_pd = parse_wiki(wiki_file)

0
ヨーロッパ
ゲーム会社一覧
任天堂
シャープ
IBM
バンダイ
ソニー
業種
南アジア
中央アジア
みずほ銀行
中央銀行
りそな銀行
日本の鉄道事業者一覧
KDDI
NTTドコモ
三井住友銀行
アフリカ
UCS
ジャレコ
コーエー
総合車両製作所
ローランド
インデックス・ホールディングス
サンリオ
10000
徳間書店
アップル (企業)
九州地方の乗合バス事業者
日亜化学工業
鉱業
紙
中国地方の乗合バス事業者
下津井電鉄
両備ホールディングス
四国地方の乗合バス事業者
アトラス (ゲーム会社)
NTTコミュニケーションズ
20000
上場
はてな (企業)
そごう
水産業
新日本海フェリー
ヤクザ
シマノ
中部地方の乗合バス事業者
イトーヨーカ堂
武富士
イズミヤ
イオン (企業)
日立製作所
カシオ計算機
イオングループ
男鹿市
イオンファンタジー
ショッピングセンター
ローソン
アイリバー
J-WAVE
南海フェリー
30000
ジャパンネット銀行
ソニー銀行
セブン銀行
労働金庫
コスモ石油
昭和シェル石油
太陽石油
ヤマト運輸
WOWOW
日本郵船
八十二銀行
保険
ジャスダック
日本発条
光岡自動車
医薬品
そごう・西武
RKB毎日放送
北アジア
キッコーマン
日野自動車
三菱自動車工業
SUBARU
TBSラジオ
三菱重工業
信託銀行
全国赤帽軽自動車運送協同組合連合会
上越市
40000
UDトラックス
いすゞ自動車
カスミ
日本交通 (大阪府)
クラシエホールディングス
ライオン (企業)
川崎重工業
佐川急便
スルガ銀行
カネカ
NBCメッシュテック
佐藤製薬
ヤマハ
トイレットペーパー
イオン北海道
北陸銀行
中国銀行 (中華人民共和国)
日本の電力会社
ブロッコリー (企業)
繊維
トマト銀行
みずほ信託銀行
So-net
佐渡汽船
コニカミノルタ
みずほ証券
みずほインベスターズ証券
アコム
ニコン
ペンタックス
富士フイルム
アサヒビール
UCC上島珈琲
サッポロビール
サッポロホールディングス
アキレス (化学工業)
サンヨー食品
大鵬薬品工業
マルハ
サントリー
コダック
50000
明治安田生命保険
名古屋証券取引所
証券
スズキ (企業)
先物取引
京都銀行
但馬銀行
セイコーエプソン
キリンビバレッジ
キリンディスティラ

In [638]:
# 各Layerの設定
import numpy as np

class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
        return None

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        h_next = np.tanh(t)
        self.cache = (x, h_prev, h_next)
        return h_next

    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache
        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis=0)
        dWh = np.dot(h_prev.T, dt)
        dh_prev = np.dot(dt, Wh.T)
        dWx = np.dot(x.T, dt)
        dx = np.dot(dt, Wx.T)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev

class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x):
        W, b = self.params
        out = np.dot(x, W) + b
        self.x = x
        return out

    def backward(self, dout):
        W, b = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0)

        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

In [639]:
# クラス分類層と損失関数の定義
# 分類機はSoftmaxなので、他クラス分類の場合はoutput_sizeをクラス数に変更するだけで良いです。

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    if t.size == y.size:
        t = t.argmax(axis=1)
    batch_size = y.shape[0]
    result = -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
    return result

def softmax(x):
    if x.ndim == 2:
        x = x - x.max(axis=1, keepdims=True)
        x = np.exp(x)
        x /= x.sum(axis=1, keepdims=True)
    elif x.ndim == 1:
        x = x - np.max(x)
        x = np.exp(x) / np.sum(np.exp(x))
    return x

class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y= None  # softmaxの出力
        self.t = None  # 教師ラベル

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)

        # 教師ラベルがone-hotベクトルの場合、正解のインデックスに変換
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)
        loss = cross_entropy_error(self.y, self.t)
        return loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1
        dx *= dout
        dx = dx / batch_size

        return dx

In [640]:
class RnnModel:
    def __init__(self, vocab_size, embed_size, hidden_size, out_size):
        # クラスの初期化
        # :param vocab_size: 単語数
        # :param embed_size: 埋め込みベクトルサイズ
        # :param hidden_size: 隠れ層サイズ
        # :param out_size: 出力層サイズ
        V, E, H, O = vocab_size, embed_size, hidden_size, out_size
        rn = np.random.randn

        self.H = H

        self.embed_W = (rn(V,E) / 100).astype("f")
        self.rnn_Wx     = (rn(E,H) / np.sqrt(E)).astype("f")
        self.rnn_Wh     = (rn(H,H) / np.sqrt(H)).astype("f")
        self.rnn_b        = np.zeros(H).astype("f")
        self.affine_W   = (rn(H,O) / np.sqrt(H)).astype("f")
        self.affine_b    = np.zeros(O).astype("f")
        # 内部状態行列
        self.h = None

        self.layers = [
            Embedding(self.embed_W),
            RNN(self.rnn_Wx, self.rnn_Wh, self.rnn_b),
            Affine(self.affine_W, self.affine_b)
        ]
        self.embedding_layer = self.layers[0]
        self.rnn_layer = self.layers[1]
        self.affine_layer = self.layers[2]
        self.loss_layer = SoftmaxWithLoss()

        self.params, self.grads = [],[]
        for layer in self.layers:
            self.params += layer.params
            self.grads  += layer.grads

    def predict(self, x):
        N,T = x.shape
        H = self.H
        x = x.T
        self.h = np.zeros((N,H), dtype="f")
        for time, word in enumerate(x):
            e = self.embedding_layer.forward(word)
            rnn_layer = RNN(self.rnn_Wx, self.rnn_Wh, self.rnn_b)
            self.h = rnn_layer.forward(e, self.h)
        y = self.affine_layer.forward(self.h)
        y = softmax(y)
        return y

    def forward(self, x, t):
        N, T = x.shape
        H = self.H
        self.x = x
        x = x.T
        self.layers = []
        self.hs = np.empty((N,T,H), dtype="f")
        self.h = np.zeros((N,H), dtype="f")
        for time, word in enumerate(x):
            e = self.embedding_layer.forward(word)
            layer = RNN(self.rnn_Wx, self.rnn_Wh, self.rnn_b)
            self.h = layer.forward(e, self.h)
            self.hs[:,time,:] = self.h
            self.layers.append(layer)
        y = self.affine_layer.forward(self.h)
        loss = self.loss_layer.forward(y, t)
        return loss

    def backward(self, dh=1):
        dh = self.loss_layer.backward(dh)
        dh = self.affine_layer.backward(dh)
        x = self.x.T
        for i, word in enumerate(reversed(x)):
            time = len(x) - 1 - i
            layer = self.layers[time]
            dx, dh = layer.backward(dh)
        dx = self.embedding_layer.backward(dx)
        return dx

    def set_state(self, h):
        self.h = h

    def reset_state(self):
        self.h = None

In [641]:
class Adam:
  def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
    self.lr = lr
    self.beta1 = beta1
    self.beta2 = beta2
    self.iter = 0
    self.m = None
    self.v = None

  def update(self, params, grads):
    if self.m is None:
      self.m, self.v = [], []
      for param in params:
        self.m.append(np.zeros_like(param))
        self.v.append(np.zeros_like(param))

    self.iter += 1
    lr_t = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)

    for i in range(len(params)):
      self.m[i] += (1 - self.beta1) * (grads[i] - self.m[i])
      self.v[i] += (1 - self.beta2) * (grads[i]**2 - self.v[i])

      params[i] -= lr_t * self.m[i] / (np.sqrt(self.v[i]) + 1e-7)

In [642]:
from janome.tokenizer import Tokenizer
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *
jt = Tokenizer()


class NumericReplaceFilter(TokenFilter):
    """
    名詞中の数(漢数字を含む)を全て0に置き換えるTokenFilterの実装
    """
    def apply(self, tokens):
        for token in tokens:
            parts = token.part_of_speech.split(',')
            if (parts[0] == '名詞' and parts[1] == '数'):
                token.surface = '0'
                token.base_form = '0'
                token.reading = 'ゼロ'
                token.phonetic = 'ゼロ'
            yield token

In [643]:
char_filters = [UnicodeNormalizeCharFilter()]
token_filters = [NumericReplaceFilter(),POSStopFilter(['記号','助詞']), LowerCaseFilter()]
tokenizer = Tokenizer()

In [644]:
import re
import numpy as np

# テキストから不要な要素を取り除く
def convert_sentence_to_words(sentence):
    stopwords = ["i", "a", "an", "the", "and", "or", "if", "is", "are", "am", "it", "this", "that", "of", "from", "in", "on"]
    sentence = sentence.lower() # 小文字化
    sentence = sentence.replace("\n", "") # 改行削除
    sentence = re.sub(re.compile("[!-\/:-@[-`{-~]"), " ", sentence) # 記号をスペースに置き換える
    sentence = sentence.split(" ") # スペースで区切る
    sentence_words = []
    for word in sentence:
        if (re.compile(r"^.*[0-9]+.*$").fullmatch(word) is not None):
            continue
        if word in stopwords:
            continue
        sentence_words.append(word)
    return sentence_words

def janome_preprocessing(text):
    text = text.lower() # 小文字化
    text = text.replace("\n", "") # 改行削除
    text = re.sub(re.compile("[!-\/:-@[-`{-~]"), "", text) # 記号をスペースに置き換える
    
    
    text_list = []

    a = Analyzer(char_filters, tokenizer, token_filters)
    
    for token in a.analyze(text):
        text_list.append(token.surface)
        
    return text_list

# 単語をIDに変換する単語辞書を生成する
def create_word_id_dict(sentence_list):
    word_to_id = {}
    for sentence in sentence_list:
        sentence_words = janome_preprocessing(sentence)
        for word in sentence_words:
            if word not in word_to_id:
                word_to_id[word] = len(word_to_id)
    return word_to_id

# 文章をID列に変換する
def convert_sentences_to_ids(sentence_list, word_to_id):
    sentence_id_vec = []
    for sentence in sentence_list:
        sentence_words = janome_preprocessing(sentence)
        sentence_ids = []
        for word in sentence_words:
            if word in word_to_id:
                sentence_ids.append(word_to_id[word])
            else:
                sentence_ids.append(-1)
        sentence_id_vec.append(sentence_ids)
    return sentence_id_vec

# 文章はそれぞれ長さが違うので、前パディングして固定長の系列にする
# 前パディングなのは、RNNの構造上、後ろにあるデータのほうが反映されやすいため
def padding_sentence(sentence_id_vec):
    max_sentence_size = 0
    for sentence_vec in sentence_id_vec:
        if max_sentence_size < len(sentence_vec):
            max_sentence_size = len(sentence_vec)
    for sentence_ids in sentence_id_vec:
        while len(sentence_ids) < max_sentence_size:
            sentence_ids.insert(0,-1)
    return sentence_id_vec

In [645]:
data_list = import_data_pd[["text","category"]].values.tolist()
import_data_list = import_data_pd["text"].values.tolist()

word_to_id = create_word_id_dict(import_data_list)
print(1)
sentence_id_vec = convert_sentences_to_ids(import_data_list, word_to_id)
print(2)
sentence_id_vec = padding_sentence(sentence_id_vec)

1
2


In [647]:
# ハイパーパラメータの設定
vocab_size = len(word_to_id)
embed_size = 200
hidden_size = 200
out_size = 33
lr = 1e-3
batch_size = 5
max_epochs = 10
data_size = len(import_data_list)
max_iters = data_size // batch_size

model = RnnModel(vocab_size, embed_size, hidden_size, out_size)
optimizer = Adam(lr)

In [648]:
# Numpy配列に変換する必要がある
x = np.array(sentence_id_vec)
t = np.array([label[1] for label in data_list])

total_loss = 0
loss_count = 0
for epoch in range(max_epochs):
    for iters in range(max_iters):
        batch_x  = x[iters*batch_size:(iters+1)*batch_size]
        batch_t =  t[iters*batch_size:(iters+1)*batch_size]
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)

        total_loss += loss
        loss_count += 1

        if (iters+1) % 10 == 0:
            avg_loss = total_loss / loss_count
            print("| epoch %d | iter %d / %d | loss %.5f" % (epoch+1, iters, max_iters, avg_loss))
            total_loss, loss_count = 0,0

| epoch 1 | iter 9 / 61 | loss 3.49090
| epoch 1 | iter 19 / 61 | loss 3.40572
| epoch 1 | iter 29 / 61 | loss 3.29775
| epoch 1 | iter 39 / 61 | loss 3.41011
| epoch 1 | iter 49 / 61 | loss 3.43927
| epoch 1 | iter 59 / 61 | loss 3.33360
| epoch 2 | iter 9 / 61 | loss 3.19496
| epoch 2 | iter 19 / 61 | loss 3.11803
| epoch 2 | iter 29 / 61 | loss 3.06897
| epoch 2 | iter 39 / 61 | loss 3.26951
| epoch 2 | iter 49 / 61 | loss 3.39138
| epoch 2 | iter 59 / 61 | loss 3.25878
| epoch 3 | iter 9 / 61 | loss 3.18923
| epoch 3 | iter 19 / 61 | loss 3.18285
| epoch 3 | iter 29 / 61 | loss 3.08554
| epoch 3 | iter 39 / 61 | loss 3.27104
| epoch 3 | iter 49 / 61 | loss 3.40139
| epoch 3 | iter 59 / 61 | loss 3.26232
| epoch 4 | iter 9 / 61 | loss 3.20553
| epoch 4 | iter 19 / 61 | loss 3.26039
| epoch 4 | iter 29 / 61 | loss 3.12203
| epoch 4 | iter 39 / 61 | loss 3.31362
| epoch 4 | iter 49 / 61 | loss 3.32813
| epoch 4 | iter 59 / 61 | loss 3.28707
| epoch 5 | iter 9 / 61 | loss 3.20855
| epo

In [649]:
test_data = [
    ["LSIをはじめとする電気機器を開発・販売する日本の企業。主にパチンコ・パチスロに組み込む、グラフィック・サウンドに関わるLSIの開発・販売に特化したファブレス企業で、直近ではグラフィック・音源・LED制御の機能を1チップに集約したLSI・AG333が主力商品となっている。"],
    ["神奈川県茅ヶ崎市に本社を置く、主に産業・研究機関向け真空装置を製造する企業である。東京証券取引所一部上場。"],
    ["神奈川県横浜市港北区に本社を置く電気計測機器、電気応用機器、電気制御装置の製造、販売を行う企業である。"],
    ["埼玉県さいたま市に本社を置く、電源3社の一角を占める電気機器メーカーである。1938年に、前身である富士電炉工業株式会社が設立され、電気炉、亜酸化銅整流器などの製造を開始した。"]
]

test_sentence_list = [s[0] for s in test_data]
test_sentence_id_vec = convert_sentences_to_ids(test_sentence_list, word_to_id)
test_sentence_id_vec = padding_sentence(test_sentence_id_vec)

for test_sentence_id in test_sentence_id_vec:
    query = np.array(test_sentence_id).reshape(1,len(test_sentence_id_vec[0]))
    result = model.predict(query)
    print(result)
    print(result.argmax())

[[0.0089961  0.00570146 0.0401389  0.09145871 0.00802984 0.01734626
  0.05861361 0.02652036 0.01988394 0.01278959 0.01876102 0.01731579
  0.00506679 0.01929841 0.01286825 0.02992295 0.04973537 0.00709611
  0.06686106 0.03083286 0.01619648 0.05119361 0.0141859  0.00660532
  0.0269461  0.00974249 0.02969437 0.17756778 0.04397963 0.02476948
  0.0203299  0.01635011 0.01520138]]
27
[[0.03896015 0.02825243 0.02692038 0.02906173 0.03316505 0.02526302
  0.03230167 0.03333958 0.0220993  0.02239187 0.0281702  0.02888313
  0.0319568  0.02629835 0.03527594 0.03060106 0.03103458 0.04809784
  0.02523643 0.03941978 0.03061692 0.024193   0.02742128 0.01674968
  0.04697725 0.03669456 0.04669637 0.02381813 0.01948339 0.01862767
  0.03671724 0.02525854 0.03001675]]
17
[[0.03737929 0.02740192 0.02204291 0.02078395 0.04631666 0.02979652
  0.03605214 0.03671641 0.02814342 0.02029867 0.03508857 0.03068232
  0.03689427 0.02699557 0.03887874 0.04049003 0.0286288  0.0366244
  0.02756658 0.02572037 0.02911617 0.

In [605]:
df = company_pd[["category","category_od"]]
a_df = df[~df.duplicated()]

In [606]:
db = pd.merge(a_df, category, on='category', how='left')

In [607]:
db

,category,category_od,中分類
0,0050,0,水産・農林業
1,1050,1,鉱業
2,2050,2,建設業
3,3050,3,食料品
4,3100,4,繊維製品
5,3150,5,パルプ・紙
6,3200,6,化学
7,3250,7,医薬品
8,3300,8,石油・石炭製品
9,3350,9,ゴム製品
